This notebook can be used for any input images (we expect this to only be needed for simulated data) that have large spreads in the count rates. 

In [2]:
import os 

from astropy.io import fits

In [3]:
def correct_image(image, upper_threshold=65000, upper_limit=65000):
    """
    Correct image for negative and overly bright pixels
    """
    img = np.copy(image)
    img[img < 0] = 0.            # neg pixs -> 0
    img[img >= upper_threshold] = upper_limit    # sat'd pixs -> 65K
    img[np.isfinite(img) == 0] = 0.

    return img

In [ ]:
# Provide path to and name of, file that needs to be corrected
path_to_file_to_be_corrected = 'my_path_to_the_file"
my_file = 'filename_of_the_file_to_be_corrected'

In [ ]:
# Open the fits file
hdu = fits.open(os.path.join(path_to_file_to_be_corrected, my_file))
hdu.info()

In [ ]:
# Correct the upper end of the counts a specified number
# upper_threshold is the limit at which you want to reassign the count rate value
# upper_limit is what the values over the upper_threshold should be set to
new_sci = correct_image(sci, upper_threshold=65000, upper_limit=65000)
new_primary = correct_image(primary, upper_threshold=65000, upper_limit=65000)

In [ ]:
# Assign corrected data to the hdu object
hdu[0].data = new_primary
hdu[1].data = new_sci

In [ ]:
# Create output filename. This is set up to give the same filename but with "_corrected" appended to the root
filename = fits_files[4].split('/')[-1]
parts = filename.split('.')
new_filename = f"{parts[0]}_corrected.{parts[1]}"
new_file = os.path.join(path_to_files, new_filename)

In [ ]:
# Write the new file
hdu.writeto(new_file)